<a href="https://colab.research.google.com/github/yuukun123/Python/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

# Tùy chỉnh để hiển thị biểu đồ đẹp hơn
sns.set(style="whitegrid")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.9 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/Final_Cleaned_Database.xlsx")
#

print(df.to_string())

In [ ]:
df, meta = pyreadstat.read_sav("/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/DelawarePainDatabase_StimulusCharacterization_forOSF.sav")

print(df.to_string())

In [1]:
import pandas as pd

# Đọc file Excel gốc
file_path = '/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/DelawarePainDatabase_SecondaryDatabase_FaceGenCharacterization.xlsx'
xls = pd.ExcelFile(file_path)

# Đọc từng sheet, thay 'NormingData_LongFormat' bằng 'FirstPassNorming'
survey_long_df = pd.read_excel(xls, 'FirstPassNorming')  # Đã sửa ở đây
face_features_df = pd.read_excel(xls, 'ExpressionAverages')  # Đã sử dụng 'ExpressionAverages'
demographics_df = pd.read_excel(xls, 'Demographics')
facegen_pro_df = pd.read_excel(xls, 'Sliders_FaceGenPro')
facegen_modeller_df = pd.read_excel(xls, 'Sliders_FaceGenModeller')

# Trích 'Expression' từ cột 'Image' trong survey_long_df
survey_long_df['Expression'] = survey_long_df['Image'].apply(lambda x: str(x).split('_')[0])

# Chuẩn hóa chữ in hoa cho khớp
survey_long_df['Expression'] = survey_long_df['Expression'].str.upper()
face_features_df['Expression'] = face_features_df['Expression'].str.upper()

# Ghép survey với face features theo Expression
merged_df = survey_long_df.merge(face_features_df, on='Expression', how='left')

# Ghép demographics vào dựa trên ResponseId
merged_df = merged_df.merge(demographics_df, on='ResponseId', how='left')

# Ghép thêm các slider vào theo Image (hoặc Expression nếu Image không khớp)
facegen_pro_df.rename(columns={'Expression': 'Image'}, inplace=True)
facegen_modeller_df.rename(columns={'Expression': 'Image'}, inplace=True)

merged_df = merged_df.merge(facegen_pro_df, on='Image', how='left', suffixes=('', '_FGP'))
merged_df = merged_df.merge(facegen_modeller_df, on='Image', how='left', suffixes=('', '_FGM'))

# Kiểm tra kết quả
print(merged_df.head())
print(merged_df.shape)

# Lưu file đã làm sạch
merged_df.to_csv('/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/Cleaned_Pain_Face_Data.csv', index=False)

print("✅ Dữ liệu đã xử lý và lưu vào 'Cleaned_Pain_Face_Data.csv'")


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


KeyError: 'Image'

In [10]:
data = pd.read_csv('/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/Cleaned_Pain_Face_Data.csv')

data.head()

# survey_long_df.head()

face_features_df.head()

,Expression,Norming Survey,PainRating,FearRating,AngerRating,DisgustRating,HappinessRating,SadnessRating,SurpriseRating,ThreatRating,...,phoneme W,phoneme aah,phoneme big aah,"phoneme ch,j,sh",phoneme ee,phoneme eh,phoneme i,phoneme oh,"phoneme ooh,Q",phoneme th
0,AJ2,FirstPass,5.052632,2.894737,2.078947,2.421053,1.210526,3.131579,2.131579,1.736842,...,.,0.08,.,.,.,.,.,.,.,.
1,AJ2,FaceGenPro,4.128205,3.384615,3.000000,2.897436,1.666667,2.923077,3.000000,.,...,.,0.08,.,.,.,.,.,.,.,.
2,AJ2,FaceGenPro_AvgBlackFace,4.044444,3.133333,2.200000,3.133333,1.200000,3.044444,2.733333,NaN,...,.,0.08,.,.,.,.,.,.,.,.
3,AK5,FirstPass,4.650000,2.225000,2.725000,2.575000,1.275000,2.275000,1.725000,2.375,...,.,0.09,.,0.5,.,.,.,0.19,.,.
4,AK5,FaceGenPro,4.428571,2.523810,2.857143,3.095238,1.928571,2.571429,2.476190,.,...,.,0.09,.,0.5,.,.,.,0.19,.,.
